In [ ]:
# | default_exp losses/class_balanced_cross_entropy_loss

# Imports

In [ ]:
# | export


from collections import Counter

import torch
from torch import nn
from torch.nn import functional as F

from vision_architectures.docstrings import populate_docstring
from vision_architectures.utils.custom_base_model import CustomBaseModel, Field

# Config

In [ ]:
# | export


class ClassBalancedCrossEntropyLossConfig(CustomBaseModel):
    num_classes: int = Field(..., description="Number of classes to weight cross entropy loss.")
    ema_decay: float = Field(
        0.99, description="Exponential moving average decay. By default 0.99 is used which has a half life of ~69 steps"
    )

# The loss function

In [ ]:
# | export


class ClassBalancedCrossEntropyLoss(nn.Module):
    @populate_docstring
    def __init__(self, config: ClassBalancedCrossEntropyLossConfig = {}, **kwargs):
        """Class-balanced cross-entropy loss with running prevalence estimation.

        This loss reweights the standard multi-class cross-entropy by the inverse of
        the observed class prevalences in the training data. Class prevalences are
        estimated online via an exponential moving average (EMA) using class counts
        from the incoming targets.

        Notes:
            - Targets must be discrete integer class indices in [0, num_classes-1].
              Probabilistic/soft labels are not supported.
            - For classes that haven't been observed yet, their prevalence is treated
              as NaN and replaced in the weight vector by the mean of observed weights
              (then clamped within 3 standard deviations to avoid extreme values).

        Args:
            config: {CONFIG_INSTANCE_DOC}
            **kwargs: {CONFIG_KWARGS_DOC}
        """
        super().__init__()

        self.config = ClassBalancedCrossEntropyLossConfig.model_validate(config | kwargs)

        # Save class prevalence percentages to weight the cross entropy loss
        self.class_prevalences = [None] * self.config.num_classes
        # Initialized with None as we don't know the initial class prevalence estimates.

    def update_class_prevalences(self, target: torch.Tensor):
        """Update the running class-prevalence estimates from a target tensor.

        The method counts class occurrences in the provided target, converts counts
        to per-batch prevalences, then updates the internal EMA-tracked prevalence
        vector for each class.

        Args:
            target: A tensor of integer class indices with any shape, typically
                (N,) or (N, ...) for segmentation. Values outside
                [0, num_classes-1] are ignored.
        """
        # Count the number of times each class is encountered
        class_counts = Counter(target.flatten().int().tolist())

        # Remove those counts that exceed the number of classes that are being tracked
        class_counts = {k: v for k, v in class_counts.items() if k < self.config.num_classes}

        # Calculate prevalence of each class
        total_count = sum(class_counts.values())
        new_prevalences = (
            {class_id: count / total_count for class_id, count in class_counts.items()} if total_count > 0 else {}
        )

        # Update current prevalences using EMA to allow for distribution shift in data
        decay = self.config.ema_decay
        for i in range(self.config.num_classes):
            if self.class_prevalences[i] is None:
                # if encountered for the first time
                if new_prevalences.get(i, 0.0) > 0.0:
                    self.class_prevalences[i] = new_prevalences.get(i, 0.0)
                # otherwise let it stay None
            else:
                # update to new value using EMA
                self.class_prevalences[i] = self.class_prevalences[i] * decay + new_prevalences.get(i, 0.0) * (
                    1 - decay
                )

    def get_class_prevalences(self, device=torch.device("cpu")) -> torch.Tensor:
        """Return the current vector of class prevalences as a tensor.

        For classes that haven't been observed yet, the corresponding entry will be
        NaN. This method does not perform any imputation or normalization beyond
        returning the current EMA state.

        Args:
            device: The device on which to place the returned tensor.

        Returns:
            Tensor of shape (num_classes,) with dtype float32 containing per-class
            prevalence estimates in [0, 1] or NaN for unseen classes.
        """

        # Get class prevalences and replace None with nan
        class_prevalences = [
            prevalence if prevalence is not None else torch.nan for prevalence in self.class_prevalences
        ]

        # Calculate weights as inverse of class prevalences
        class_prevalences = torch.tensor(class_prevalences, dtype=torch.float32, device=device)

        return class_prevalences

    def get_class_weights(self, device=torch.device("cpu")) -> torch.Tensor:
        """Compute per-class weights as the inverse of prevalences with safeguards.

        Steps:
            1) Convert current EMA prevalences to a tensor with NaNs for unseen classes.
            2) Take the inverse to obtain raw weights (higher weight for rarer classes).
            3) Replace NaNs by the mean of observed weights to avoid biasing toward
               unseen classes, then clamp to mean ± 3·std to prevent extreme values.
            4) Renormalize weights to sum to num_classes (so the average weight is 1).

        Args:
            device: The device on which to place the returned tensor.

        Returns:
            Tensor of shape (num_classes,) containing normalized class weights.
        """

        # Get class prevalences
        class_prevalences = self.get_class_prevalences(device)

        # If there were no class prevalences, then all weights will be nan; handle this
        if class_prevalences.isnan().all():
            class_prevalences = class_prevalences.nan_to_num(1.0)

        # Calculate weights as inverse of class prevalences
        weights = 1 / class_prevalences

        # Substitute nan values with mean and clamp weights to a limit
        # Assumption: all classes are visited at least once in the dataset
        mu, std = weights.nanmean(), weights[~weights.isnan()].std()
        weights = weights.nan_to_num(mu)
        weights = weights.clamp(mu - 3 * std, mu + 3 * std)

        # Add failsafe just in case nans are still present
        weights = weights.nan_to_num(1.0)

        # Normalize weights to sum to self.config.num_classes
        weights = self.config.num_classes * weights / weights.sum()

        return weights

    def forward(
        self,
        input: torch.Tensor,
        target: torch.Tensor,
        update_class_prevalences: bool = True,
        return_class_weights: bool = False,
        *args,
        **kwargs
    ) -> torch.Tensor | tuple[torch.Tensor, torch.Tensor]:
        """Compute class-balanced cross entropy.

        This updates the internal class-prevalence EMA using the provided targets,
        then computes cross-entropy with a weight vector derived from the current
        prevalences.

        Args:
            input: Logits of shape (N, C, ...) where C == num_classes. Any extra spatial
                dimensions (e.g., H, W, D) are supported as long as `target` is broadcastable
                to the same non-channel shape expected by torch.nn.functional.cross_entropy.
            target: Integer class indices with shape matching input without the channel
                dimension, e.g., (N, ...) with values in [0, C-1].
            update_class_prevalences: If True, update the internal class prevalence estimates
                using the provided targets.
            return_class_weights: If True, also return the per-class weight tensor used
                for this call.
            *args, **kwargs: Additional keyword args forwarded to F.cross_entropy
                (e.g., reduction='mean').

        Returns:
            If return_class_weights is False: a scalar tensor loss.
            If True: a tuple (loss, class_weights).
        """
        # Update class prevalences
        if update_class_prevalences:
            self.update_class_prevalences(target)

        # Get class weights
        class_weights = self.get_class_weights(input.device)

        # Calculate loss
        loss = F.cross_entropy(input, target, weight=class_weights, *args, **kwargs)

        if return_class_weights:
            return loss, class_weights
        return loss

In [ ]:
# Regular test


test = ClassBalancedCrossEntropyLoss(num_classes=3)

for _ in range(10):
    example_input = torch.rand(4, 3)
    example_target = torch.randint(0, 3, (4,))
    loss, class_weights = test(example_input, example_target, return_class_weights=True)
    print(loss, class_weights)

tensor(1.2635) tensor([1.2000, 0.6000, 1.2000])
tensor(1.1835) tensor([1.1798, 0.6047, 1.2155])
tensor(1.2358) tensor([1.1836, 0.6096, 1.2069])
tensor(1.0405) tensor([1.1837, 0.6095, 1.2068])
tensor(1.1366) tensor([1.1957, 0.6094, 1.1949])
tensor(1.1001) tensor([1.2076, 0.6093, 1.1831])
tensor(1.0922) tensor([1.2194, 0.6090, 1.1715])
tensor(1.0698) tensor([1.2258, 0.6185, 1.1557])
tensor(1.1738) tensor([1.2170, 0.6234, 1.1596])
tensor(1.0539) tensor([1.2084, 0.6282, 1.1635])


In [ ]:
# Test where last two classes are never encountered

test = ClassBalancedCrossEntropyLoss(num_classes=5)

for _ in range(10):
    example_input = torch.rand(4, 5)
    example_target = torch.randint(0, 3, (4,))
    loss, class_weights = test(example_input, example_target, return_class_weights=True)
    print(loss, class_weights)

tensor(1.3756) tensor([1.2000, 0.6000, 1.2000, 1.0000, 1.0000])
tensor(1.7763) tensor([1.2036, 0.6048, 1.1916, 1.0000, 1.0000])
tensor(1.5222) tensor([1.2070, 0.6096, 1.1834, 1.0000, 1.0000])
tensor(1.7942) tensor([1.2069, 0.6095, 1.1836, 1.0000, 1.0000])
tensor(1.6321) tensor([1.2188, 0.6092, 1.1720, 1.0000, 1.0000])
tensor(1.5927) tensor([1.2219, 0.6139, 1.1642, 1.0000, 1.0000])
tensor(1.3780) tensor([1.2132, 0.6188, 1.1681, 1.0000, 1.0000])
tensor(1.7535) tensor([1.2280, 0.6231, 1.1488, 1.0000, 1.0000])
tensor(1.9232) tensor([1.2277, 0.6229, 1.1493, 1.0000, 1.0000])
tensor(1.5832) tensor([1.2220, 0.6326, 1.1454, 1.0000, 1.0000])


In [ ]:
# Don't update prevalences

test = ClassBalancedCrossEntropyLoss(num_classes=3)

for _ in range(3):
    example_input = torch.rand(4, 3)
    example_target = torch.randint(0, 3, (4,))
    loss, class_weights = test(example_input, example_target, update_class_prevalences=False, return_class_weights=True)
    print(loss, class_weights)

tensor(1.0290) tensor([1., 1., 1.])
tensor(1.0518) tensor([1., 1., 1.])
tensor(0.9804) tensor([1., 1., 1.])


# nbdev

In [ ]:
!nbdev_export